In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, BatchNormalization
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from tensorflow.keras import optimizers
import numpy as np

In [2]:
(trainX, trainy), (testX, testy) = cifar10.load_data()
# print to make sure we have the correct shapes + number of images for training
print("number of train pictures:", trainX.shape)
print("number of trained picture values:", trainy.shape)
# divide by 255 to make [0,255] into [0,1] + print to make sure!
trainy = tf.keras.utils.to_categorical(trainy,10)
testy = tf.keras.utils.to_categorical(testy,10)
trainX = trainX/255.0
testX = testX/255.0

number of train pictures: (50000, 32, 32, 3)
number of trained picture values: (50000, 1)


In [3]:
import tensorflow_model_optimization as tfmot

LastValueQuantizer = tfmot.quantization.keras.quantizers.LastValueQuantizer
MovingAverageQuantizer = tfmot.quantization.keras.quantizers.MovingAverageQuantizer

class ModifiedDenseQuantizeConfig(tfmot.quantization.keras.QuantizeConfig):
    def get_weights_and_quantizers(self, layer):
      return [(layer.kernel, LastValueQuantizer(num_bits=8, symmetric=True, narrow_range=False, per_axis=False))]

    def get_activations_and_quantizers(self, layer):
      return [(layer.activation, MovingAverageQuantizer(num_bits=8, symmetric=False, narrow_range=False, per_axis=False))]

    def set_quantize_weights(self, layer, quantize_weights):
      # Add this line for each item returned in `get_weights_and_quantizers`
      # , in the same order
      layer.kernel = quantize_weights[0]

    def set_quantize_activations(self, layer, quantize_activations):
      # Add this line for each item returned in `get_activations_and_quantizers`
      # , in the same order.
      layer.activation = quantize_activations[0]

    # Configure how to quantize outputs (may be equivalent to activations).
    def get_output_quantizers(self, layer):
      return []

    def get_config(self):
      return {}

### Quantizing vgg-16

In [6]:
annotate = tfmot.quantization.keras.quantize_annotate_layer

quant_vgg16 = tf.keras.Sequential()
    # Only annotated layers will be quantized
    
#block-1
quant_vgg16.add(annotate(Conv2D(input_shape=(32,32,3),
                    filters=64,kernel_size=(3,3),
                    padding="same", 
                    activation="relu",
                    name='block1_conv1'), quantize_config=ModifiedDenseQuantizeConfig()))
quant_vgg16.add(BatchNormalization())
quant_vgg16.add(Dropout(0.3))   

quant_vgg16.add(annotate(Conv2D(filters=64,
                    kernel_size=(3,3),
                    padding="same", 
                    activation="relu",
                    name='block1_conv2'),
                    quantize_config=ModifiedDenseQuantizeConfig()))
quant_vgg16.add(BatchNormalization())
quant_vgg16.add(Dropout(0.4))

quant_vgg16.add(MaxPool2D(pool_size=(2,2), strides=(2,2), name='block1_pool'))


#block-2
quant_vgg16.add(annotate(Conv2D(filters=128, 
                    kernel_size=(3,3), 
                    padding="same", 
                    activation="relu",
                    name='block2_conv1'),
                    quantize_config=ModifiedDenseQuantizeConfig()))
quant_vgg16.add(BatchNormalization())
quant_vgg16.add(Dropout(0.4))

quant_vgg16.add(annotate(Conv2D(filters=128, 
                    kernel_size=(3,3), 
                    padding="same", 
                    activation="relu",
                    name='block2_conv2'),
                    quantize_config=ModifiedDenseQuantizeConfig()))
quant_vgg16.add(BatchNormalization())

quant_vgg16.add(MaxPool2D(pool_size=(2,2),strides=(2,2), name='block2_pool'))

#block-3
quant_vgg16.add(annotate(Conv2D(filters=256, 
                    kernel_size=(3,3), 
                    padding="same", 
                    activation="relu",
                    name='block3_conv1'),
                    quantize_config=ModifiedDenseQuantizeConfig()))
quant_vgg16.add(BatchNormalization())
quant_vgg16.add(Dropout(0.4))

quant_vgg16.add(annotate(Conv2D(filters=256, 
                    kernel_size=(3,3), 
                    padding="same", 
                    activation="relu",
                    name='block3_conv2'),
                    quantize_config=ModifiedDenseQuantizeConfig()))
quant_vgg16.add(BatchNormalization())
quant_vgg16.add(Dropout(0.4))

quant_vgg16.add(annotate(Conv2D(filters=256, 
                    kernel_size=(3,3), 
                    padding="same", 
                    activation="relu",
                    name='block3_conv3'),
                    quantize_config=ModifiedDenseQuantizeConfig()))
quant_vgg16.add(BatchNormalization())

quant_vgg16.add(MaxPool2D(pool_size=(2,2),strides=(2,2), name='block3_pool'))

#block-4
quant_vgg16.add(annotate(Conv2D(filters=512, 
                    kernel_size=(3,3), 
                    padding="same", 
                    activation="relu",
                    name='block4_conv1'),
                    quantize_config=ModifiedDenseQuantizeConfig()))
quant_vgg16.add(BatchNormalization())
quant_vgg16.add(Dropout(0.4))

quant_vgg16.add(annotate(Conv2D(filters=512, 
                    kernel_size=(3,3), 
                    padding="same", 
                    activation="relu",
                    name='block4_conv2'),
                    quantize_config=ModifiedDenseQuantizeConfig()))
quant_vgg16.add(BatchNormalization())
quant_vgg16.add(Dropout(0.4))

quant_vgg16.add(annotate(Conv2D(filters=512, 
                    kernel_size=(3,3), 
                    padding="same", 
                    activation="relu",
                    name='block4_conv3'),
                    quantize_config=ModifiedDenseQuantizeConfig()))
quant_vgg16.add(BatchNormalization())

quant_vgg16.add(MaxPool2D(pool_size=(2,2),strides=(2,2), name='block4_pool'))

#block-5
quant_vgg16.add(annotate(Conv2D(filters=512, 
                    kernel_size=(3,3), 
                    padding="same", 
                    activation="relu",
                    name='block5_conv1'),
                    quantize_config=ModifiedDenseQuantizeConfig()))
quant_vgg16.add(Dropout(0.4))
quant_vgg16.add(annotate(Conv2D(filters=512, 
                    kernel_size=(3,3), 
                    padding="same", 
                    activation="relu",
                    name='block5_conv2'),
                    quantize_config=ModifiedDenseQuantizeConfig()))
quant_vgg16.add(BatchNormalization())
quant_vgg16.add(Dropout(0.4))

quant_vgg16.add(annotate(Conv2D(filters=512, 
                    kernel_size=(3,3), 
                    padding="same", 
                    activation="relu",
                    name='block5_conv3'),
                    quantize_config=ModifiedDenseQuantizeConfig()))
quant_vgg16.add(BatchNormalization())

quant_vgg16.add(MaxPool2D(pool_size=(2,2),strides=(2,2), name='block5_pool'))

#fc1, fc2 and predictions
quant_vgg16.add(Dropout(0.5))
quant_vgg16.add(annotate(Flatten(name='flatten')))
quant_vgg16.add(annotate(Dense(units=512,activation="relu",name='fc1'), quantize_config=ModifiedDenseQuantizeConfig()))
quant_vgg16.add(BatchNormalization())

quant_vgg16.add(Dropout(0.5))
quant_vgg16.add(Dense(units=10, activation="softmax",name='predictions'))    
  

quantize_scope = tfmot.quantization.keras.quantize_scope

# `quantize_apply` requires mentioning `DefaultDenseQuantizeConfig` with `quantize_scope`
# as well as the custom Keras layer.
with quantize_scope(
  {'ModifiedDenseQuantizeConfig':ModifiedDenseQuantizeConfig}):
  # Use `quantize_apply` to actually make the model quantization aware.
  vgg_quant_model = tfmot.quantization.keras.quantize_apply(quant_vgg16)
    
vgg_quant_model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_layer (QuantizeLaye (None, 32, 32, 3)         3         
_________________________________________________________________
quant_block1_conv1 (Quantize (None, 32, 32, 64)        1797      
_________________________________________________________________
batch_normalization_13 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
dropout_11 (Dropout)         (None, 32, 32, 64)        0         
_________________________________________________________________
quant_block1_conv2 (Quantize (None, 32, 32, 64)        36933     
_________________________________________________________________
batch_normalization_14 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
dropout_12 (Dropout)         (None, 32, 32, 64)       

In [7]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD

opt = SGD(learning_rate=0.1, decay=1e-6, momentum=0.9, nesterov=True)
# Compile the model
vgg_quant_model.compile(optimizer=opt, loss=tf.keras.losses.categorical_crossentropy,metrics=['accuracy'])

# Fit data to model
vgg_quant_model.fit(trainX, trainy,
          batch_size=50,
          epochs=100,
          verbose=1,
          validation_split=0.2)

Epoch 1/100
800/800 [==============================] - 19s 19ms/step - loss: 2.4709 - accuracy: 0.1472 - val_loss: 2.5450 - val_accuracy: 0.1058
Epoch 2/100
800/800 [==============================] - 15s 19ms/step - loss: 2.1276 - accuracy: 0.1933 - val_loss: 2.7973 - val_accuracy: 0.1202
Epoch 3/100
800/800 [==============================] - 15s 19ms/step - loss: 2.2015 - accuracy: 0.1767 - val_loss: 3.0607 - val_accuracy: 0.1038
Epoch 4/100
800/800 [==============================] - 15s 19ms/step - loss: 2.1505 - accuracy: 0.1823 - val_loss: 3.6451 - val_accuracy: 0.1019
Epoch 5/100
800/800 [==============================] - 15s 19ms/step - loss: 2.0494 - accuracy: 0.2053 - val_loss: 3.6542 - val_accuracy: 0.1027
Epoch 6/100
800/800 [==============================] - 15s 19ms/step - loss: 1.9976 - accuracy: 0.2194 - val_loss: 3.1324 - val_accuracy: 0.1060
Epoch 7/100
800/800 [==============================] - 15s 19ms/step - loss: 1.9596 - accuracy: 0.2344 - val_loss: 3.5248 - val_ac

800/800 [==============================] - 15s 19ms/step - loss: 0.8429 - accuracy: 0.7165 - val_loss: 2.9117 - val_accuracy: 0.2900
Epoch 58/100
800/800 [==============================] - 15s 19ms/step - loss: 0.8211 - accuracy: 0.7296 - val_loss: 2.0459 - val_accuracy: 0.4020
Epoch 59/100
800/800 [==============================] - 15s 19ms/step - loss: 0.8079 - accuracy: 0.7350 - val_loss: 2.0349 - val_accuracy: 0.4189
Epoch 60/100
800/800 [==============================] - 15s 19ms/step - loss: 0.7964 - accuracy: 0.7364 - val_loss: 2.3364 - val_accuracy: 0.3566
Epoch 61/100
800/800 [==============================] - 15s 19ms/step - loss: 0.7758 - accuracy: 0.7445 - val_loss: 2.2529 - val_accuracy: 0.3749
Epoch 62/100
800/800 [==============================] - 15s 19ms/step - loss: 0.7642 - accuracy: 0.7487 - val_loss: 2.4145 - val_accuracy: 0.3757
Epoch 63/100
800/800 [==============================] - 15s 19ms/step - loss: 0.7652 - accuracy: 0.7494 - val_loss: 2.0597 - val_accuracy

In [8]:
vgg_quant_model.save_weights('cifar10vgg_quant8.h5')

In [9]:
score = vgg_quant_model.evaluate(testX, testy, verbose=1)
print("Test loss {:.4f}, accuracy {:.2f}%".format(score[0], score[1] * 100))

313/313 [==============================] - 2s 7ms/step - loss: 2.0701 - accuracy: 0.5220
Test loss 2.0701, accuracy 52.20%


In [ ]:
52